<a href="https://colab.research.google.com/github/ChessEngineUS/protein-folding-visualizer/blob/main/notebooks/colab_casp15_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏆 CASP15 Benchmark - Google Colab Edition

**Official CASP15 Evaluation with AlphaFold 3 & Boltz-2**

This notebook runs the complete CASP15 benchmark evaluation in Google Colab with GPU acceleration.

### Requirements:
- 💻 Colab Pro or Pro+ recommended (for A100 GPU)
- ⏱️ Runtime: 2-4 hours for full evaluation
- 💾 Storage: ~60GB for model weights

### Novel Contributions:
1. ✅ First unified AF3 + Boltz-2 benchmark
2. ✅ Ensemble uncertainty quantification
3. ✅ Confidence-accuracy correlation
4. ✅ Computational efficiency analysis

---

In [ ]:
# Check GPU
import torch
print(f'GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None"}')
print(f'Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB' if torch.cuda.is_available() else 'No GPU')

# Clone repo
!git clone https://github.com/ChessEngineUS/protein-folding-visualizer.git
%cd protein-folding-visualizer

# Install
!pip install -q -r requirements.txt
!pip install -e .

print('✅ Setup complete!')

In [ ]:
# Download model weights (~30-60 min)
# Only run once!

!python scripts/download_models.py --alphafold3 --boltz2

print('✅ Models downloaded!')

In [ ]:
from src.alphafold3 import AlphaFold3Predictor
from src.evaluation import BenchmarkSuite

# Initialize
predictor = AlphaFold3Predictor(model_dir='./models/alphafold3')
benchmark = BenchmarkSuite(output_dir='./benchmarks/casp15')

# Run benchmark
results = benchmark.run_casp15_benchmark(
    predictor=predictor,
    targets=['T1104', 'T1124']  # Easy + Medium
)

# Generate report
report = benchmark.generate_benchmark_report()

print(f'\n✅ Benchmark complete!')
print(f'Report: {report}')

## 📊 View Results

Results are saved and can be visualized below.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load results
df = pd.DataFrame([
    {
        'Target': r.target_id,
        'GDT_TS': r.gdt_ts,
        'TM-score': r.tm_score,
        'lDDT': r.lddt
    }
    for r in results
])

print(df)

# Plot
df.plot(x='Target', y=['GDT_TS', 'TM-score', 'lDDT'], kind='bar', figsize=(12, 6))
plt.title('CASP15 Performance')
plt.ylabel('Score')
plt.legend()
plt.show()